In [1]:
from app import * # import data_model.py module with all functions included.
try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)

Database successfully created.


/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

from flask import Flask, render_template, redirect, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))


class RegisterForm(FlaskForm):
    email = StringField("邮箱", validators=[InputRequired(), Email(message="无效的邮箱"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("用户名", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "用户名"})
    password = PasswordField("密码", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("注册")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("用户名已存在，请重新输入.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("邮箱已存在，请重新输入.")


class LoginForm(FlaskForm):
    username = StringField("用户名", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("密码", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("登录")


@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))
    
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route("/")
def index():
    return render_template('index.html')


@app.route("/Shop")
def Shop():
    #df = pd.read_csv("猫咖-点评.csv")
    #list = df.to_dict('records')
    list=[{'Unnamed: 0': 0, '店铺名称': '猫不理·猫咪咖啡·布偶猫舍', '店铺星级（50=5星）': 45, '人均消费': '54.0', '所在区域': '海琴广场', '团购': '190元 四人撸猫套餐（十六岁以下不接待）'}, {'Unnamed: 0': 1, '店铺名称': '喵小院猫咖The Cat Coffee(五四广场店)', '店铺星级（50=5星）': 45, '人均消费': '36.0', '所在区域': '万象城', '团购': '39.9元 晚场17-21点单人撸猫套餐'}, {'Unnamed: 0': 2, '店铺名称': '春夏秋猫咖啡·猫咪主题咖啡馆', '店铺星级（50=5星）': 45, '人均消费': '34.0', '所在区域': '长江路沿线', '团购': '无'}, {'Unnamed: 0': 3, '店铺名称': '狗窝窝Gowow·羊驼·猫咖狗咖', '店铺星级（50=5星）': 45, '人均消费': '76.0', '所在区域': '万象城', '团购': '88元 1-3楼单人通票·撸狗·撸羊驼·撸鸭·撸猫'}, {'Unnamed: 0': 4, '店铺名称': '六月猫咖·英短猫舍', '店铺星级（50=5星）': 50, '人均消费': '43.0', '所在区域': '台东步行街', '团购': '39.9元 撸猫单人餐，提供免费WiFi'}, {'Unnamed: 0': 5, '店铺名称': '三月猫咖', '店铺星级（50=5星）': 45, '人均消费': '45.0', '所在区域': '新都心', '团购': '99元 双人套餐，包间免费'}, {'Unnamed: 0': 6, '店铺名称': 'Coffee Meow 米奥猫咖', '店铺星级（50=5星）': 40, '人均消费': '29.0', '所在区域': '正阳路', '团购': '19元 喵星人单人饮品，有赠品'}, {'Unnamed: 0': 7, '店铺名称': '柴圆滚滚猫咪咖啡', '店铺星级（50=5星）': 45, '人均消费': '40.0', '所在区域': '中央商务区', '团购': '39.9元 单人超值撸喵体验券'}, {'Unnamed: 0': 8, '店铺名称': '布偶猫咖(台东步行街店)', '店铺星级（50=5星）': 35, '人均消费': '25.0', '所在区域': '台东步行街', '团购': '36.9元 单人撸猫体验，包间免费'}, {'Unnamed: 0': 9, '店铺名称': 'Café de DARI 达芮.猫咖啡馆(新城吾悦店)', '店铺星级（50=5星）': 40, '人均消费': '52.0', '所在区域': '海上嘉年华', '团购': '无'}, {'Unnamed: 0': 10, '店铺名称': '一树喵生撸猫下午茶猫咖啡馆', '店铺星级（50=5星）': 40, '人均消费': '46.0', '所在区域': '维客广场', '团购': '68元 单人限定静享下午茶时光套餐'}, {'Unnamed: 0': 11, '店铺名称': '福至猫吉·海景猫咖', '店铺星级（50=5星）': 40, '人均消费': '33.0', '所在区域': '融创茂', '团购': '无'}, {'Unnamed: 0': 12, '店铺名称': '猫在云端•猫咪咖啡•布偶猫舍', '店铺星级（50=5星）': 30, '人均消费': '无', '所在区域': '丽达购物广场', '团购': '72.8元 双人撸猫套餐5选2'}, {'Unnamed: 0': 13, '店铺名称': '喵町物语·宅猫咖', '店铺星级（50=5星）': 45, '人均消费': '36.0', '所在区域': '湛山/太平角', '团购': '78元 双人撸猫体验'}, {'Unnamed: 0': 14, '店铺名称': '杨小喵猫咖', '店铺星级（50=5星）': 45, '人均消费': '31.0', '所在区域': '维客广场', '团购': '55元 双人人套餐，包间免费'}, {'Unnamed: 0': 15, '店铺名称': '柴岛日记·柴犬狗咖·猫咖·海景咖啡店', '店铺星级（50=5星）': 40, '人均消费': '44.0', '所在区域': '五四广场商圈', '团购': '128元 双人暖心撸猫撸狗'}, {'Unnamed: 0': 16, '店铺名称': 'Gowow狗窝窝·楼上有猫咖(五四广场店)', '店铺星级（50=5星）': 40, '人均消费': '69.0', '所在区域': '万象城', '团购': '无'}, {'Unnamed: 0': 17, '店铺名称': '妙喵屋猫咖', '店铺星级（50=5星）': 35, '人均消费': '29.0', '所在区域': '青岛路', '团购': '49.9元 撸猫双人套餐，提供免费WiFi'}, {'Unnamed: 0': 18, '店铺名称': '海边的猫和咖啡馆', '店铺星级（50=5星）': 45, '人均消费': '50.0', '所在区域': '湛山/太平角', '团购': '无'}, {'Unnamed: 0': 19, '店铺名称': '你和猫&猫咖', '店铺星级（50=5星）': 35, '人均消费': '43.0', '所在区域': '石老人海水浴场商圈', '团购': '[猫咖]71.9元 双人撸猫券'}, {'Unnamed: 0': 24, '店铺名称': '一只狗的猫咖馆·爬宠·阿拉斯加', '店铺星级（50=5星）': 40, '人均消费': '42.0', '所在区域': '中央商务区', '团购': '45.5元 单人撸猫撸狗·爬宠互动体验（谢绝16岁以下儿童）'}, {'Unnamed: 0': 25, '店铺名称': 'Sweet cat·星愿猫咖', '店铺星级（50=5星）': 35, '人均消费': '31.0', '所在区域': '长江路沿线', '团购': '25.8元 单人撸猫套餐，提供免费WiFi'}, {'Unnamed: 0': 26, '店铺名称': '小巫咖啡馆', '店铺星级（50=5星）': 45, '人均消费': '54.0', '所在区域': '湛山/太平角', '团购': '无'}, {'Unnamed: 0': 27, '店铺名称': '晴天猫星球·猫咪咖啡', '店铺星级（50=5星）': 45, '人均消费': '69.0', '所在区域': '新都心', '团购': '68.8元 暖冬单人餐❤，提供免费WiFi'}, {'Unnamed: 0': 28, '店铺名称': '朵猫猫撸猫咖啡馆', '店铺星级（50=5星）': 45, '人均消费': '44.0', '所在区域': '新都心', '团购': '49.9元 单人撸猫套餐（14周岁以下不接待）'}]
    return render_template('Shop.html', entries = list, encoding="utf-8")


@app.route("/Contact")
def Contact():
    return render_template('Contact.html')


@app.route("/Guide")
def Guide():
    return render_template('Guide.html')


################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################

# APPLICATION TEST RUN AT PORT 9003

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 8896)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:8896/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 16:17:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:17:16] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:17:16] "GET /static/js/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2021 16:17:16] "GET /static/assets/img/intro.jpg HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:17:18] "GET /static/assets/img/bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2021 16:18:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:18:15] "GET /static/assets/img/bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:18:18] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:18

In [12]:
df = pd.read_csv("猫咖-点评.csv")
df


Unnamed: 0                         店铺名称  店铺星级（50=5星）  人均消费       所在区域  \
0            0                猫不理·猫咪咖啡·布偶猫舍           45  54.0       海琴广场   
1            1   喵小院猫咖The Cat Coffee(五四广场店)           45  36.0        万象城   
2            2               春夏秋猫咖啡·猫咪主题咖啡馆           45  34.0      长江路沿线   
3            3             狗窝窝Gowow·羊驼·猫咖狗咖           45  76.0        万象城   
4            4                    六月猫咖·英短猫舍           50  43.0      台东步行街   
5            5                         三月猫咖           45  45.0        新都心   
6            6             Coffee Meow 米奥猫咖           40  29.0        正阳路   
7            7                     柴圆滚滚猫咪咖啡           45  40.0      中央商务区   
8            8                 布偶猫咖(台东步行街店)           35  25.0      台东步行街   
9            9  Café de DARI 达芮.猫咖啡馆(新城吾悦店)           40  52.0      海上嘉年华   
10          10                一树喵生撸猫下午茶猫咖啡馆           40  46.0       维客广场   
11          11                    福至猫吉·海景猫咖           40  33.0        融创茂   
12          12               猫在云端•猫咪咖啡•布偶猫舍           30     无     丽达购物广场   
13          13                     喵町物语·宅猫咖           45  36.0     湛山/太平角   
14          14                        杨小喵猫咖           45  31.0       维客广场   
15          15           柴岛日记·柴犬狗咖·猫咖·海景咖啡店           40  44.0     五四广场商圈   
16          16        Gowow狗窝窝·楼上有猫咖(五四广场店)           40  69.0        万象城   
17          17                        妙喵屋猫咖           35  29.0        青岛路   
18          18                     海边的猫和咖啡馆           45  50.0     湛山/太平角   
19          19                       你和猫&猫咖           35  43.0  石老人海水浴场商圈   
20          24              一只狗的猫咖馆·爬宠·阿拉斯加           40  42.0      中央商务区   
21          25               Sweet cat·星愿猫咖           35  31.0      长江路沿线   
22          26                        小巫咖啡馆           45  54.0     湛山/太平角   
23          27                   晴天猫星球·猫咪咖啡           45  69.0        新都心   
24          28                     朵猫猫撸猫咖啡馆           45  44.0        新都心   

                                团购  
0            190元 四人撸猫套餐（十六岁以下不接待）  
1             39.9元 晚场17-21点单人撸猫套餐  
2                                无  
3        88元 1-3楼单人通票·撸狗·撸羊驼·撸鸭·撸猫  
4             39.9元 撸猫单人餐，提供免费WiFi  
5                    99元 双人套餐，包间免费  
6                  19元 喵星人单人饮品，有赠品  
7                  39.9元 单人超值撸喵体验券  
8                36.9元 单人撸猫体验，包间免费  
9                                无  
10               68元 单人限定静享下午茶时光套餐  
11                               无  
12                 72.8元 双人撸猫套餐5选2  
13                      78元 双人撸猫体验  
14                  55元 双人人套餐，包间免费  
15                   128元 双人暖心撸猫撸狗  
16                               无  
17           49.9元 撸猫双人套餐，提供免费WiFi  
18                               无  
19                 [猫咖]71.9元 双人撸猫券  
20  45.5元 单人撸猫撸狗·爬宠互动体验（谢绝16岁以下儿童）  
21           25.8元 单人撸猫套餐，提供免费WiFi  
22                               无  
23           68.8元 暖冬单人餐❤，提供免费WiFi  
24         49.9元 单人撸猫套餐（14周岁以下不接待）

In [24]:
list = df.to_dict('records')
print(list)

[{'Unnamed: 0': 0, '店铺名称': '猫不理·猫咪咖啡·布偶猫舍', '店铺星级（50=5星）': 45, '人均消费': '54.0', '所在区域': '海琴广场', '团购': '190元 四人撸猫套餐（十六岁以下不接待）'}, {'Unnamed: 0': 1, '店铺名称': '喵小院猫咖The Cat Coffee(五四广场店)', '店铺星级（50=5星）': 45, '人均消费': '36.0', '所在区域': '万象城', '团购': '39.9元 晚场17-21点单人撸猫套餐'}, {'Unnamed: 0': 2, '店铺名称': '春夏秋猫咖啡·猫咪主题咖啡馆', '店铺星级（50=5星）': 45, '人均消费': '34.0', '所在区域': '长江路沿线', '团购': '无'}, {'Unnamed: 0': 3, '店铺名称': '狗窝窝Gowow·羊驼·猫咖狗咖', '店铺星级（50=5星）': 45, '人均消费': '76.0', '所在区域': '万象城', '团购': '88元 1-3楼单人通票·撸狗·撸羊驼·撸鸭·撸猫'}, {'Unnamed: 0': 4, '店铺名称': '六月猫咖·英短猫舍', '店铺星级（50=5星）': 50, '人均消费': '43.0', '所在区域': '台东步行街', '团购': '39.9元 撸猫单人餐，提供免费WiFi'}, {'Unnamed: 0': 5, '店铺名称': '三月猫咖', '店铺星级（50=5星）': 45, '人均消费': '45.0', '所在区域': '新都心', '团购': '99元 双人套餐，包间免费'}, {'Unnamed: 0': 6, '店铺名称': 'Coffee Meow 米奥猫咖', '店铺星级（50=5星）': 40, '人均消费': '29.0', '所在区域': '正阳路', '团购': '19元 喵星人单人饮品，有赠品'}, {'Unnamed: 0': 7, '店铺名称': '柴圆滚滚猫咪咖啡', '店铺星级（50=5星）': 45, '人均消费': '40.0', '所在区域': '中央商务区', '团购': '39.9元 单人超值撸喵体验券'}, {'Unnamed: 0': 8, '店铺名称': '布